In [2]:
!gcloud dataproc clusters list --region us-central1


NAME          PLATFORM  WORKER_COUNT  PREEMPTIBLE_WORKER_COUNT  STATUS   ZONE           SCHEDULED_DELETE
cluster-bd87  GCE       4                                       RUNNING  us-central1-a


In [3]:
!pip install -q google-cloud-storage==1.43.0
!pip install -q graphframes

In [4]:
!pip install -q pyspark
!pip install -U -q PyDrive
!apt-get update -qq
!apt install openjdk-8-jdk-headless -qq

Package openjdk-8-jdk-headless is not available, but is referred to by another package.
This may mean that the package is missing, has been obsoleted, or
is only available from another source

E: Package 'openjdk-8-jdk-headless' has no installation candidate


In [5]:
import pyspark
import sys
from collections import Counter, OrderedDict, defaultdict
import itertools
from itertools import islice, count, groupby
import pandas as pd
import os
import re
from operator import itemgetter
import nltk
from nltk.stem.porter import *
from nltk.corpus import stopwords
from time import time
from pathlib import Path
import pickle
import pandas as pd
from google.cloud import storage

import hashlib
def _hash(s):
    return hashlib.blake2b(bytes(s, encoding='utf8'), digest_size=5).hexdigest()

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [6]:
!ls -l /usr/lib/spark/jars/graph*

-rw-r--r-- 1 root root 247882 Jan  2 12:47 /usr/lib/spark/jars/graphframes-0.8.2-spark3.1-s_2.12.jar


In [7]:
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf, SparkFiles
from pyspark.sql import SQLContext
from graphframes import *

In [8]:
# Put your bucket name below and make sure you can access it without an error
bucket_name = '316399401' 
client = storage.Client()
blobs = client.list_blobs(bucket_name)

In [20]:
full_path = "gs://wikidata_preprocessed/*"
parquetFile = spark.read.parquet(full_path)
doc_text_pairs = parquetFile.select("text","title","id").rdd


In [43]:
def tokenize(text):
  #TODO probablly need some fixes
    """
    This function aims in tokenize a text into a list of tokens. Moreover, it filter stopwords.
    
    Parameters:
    -----------
    text: string , represting the text to tokenize.    
    
    Returns:
    -----------
    list of tokens (e.g., list of tokens).
    """
    list_of_tokens =  [token.group() for token in RE_WORD.finditer(text.lower()) if token.group() not in all_stopwords]    
    return list_of_tokens

In [44]:
stopwords_frozen = frozenset(stopwords.words('english'))
corpus_stopwords = ["category", "references", "also", "external", "links", 
                    "may", "first", "see", "history", "people", "one", "two", 
                    "part", "thumb", "including", "second", "following", 
                    "many", "however", "would", "became"]

all_stopwords = stopwords_frozen.union(corpus_stopwords)
RE_WORD = re.compile(r"""[\#\@\w](['\-]?\w){2,24}""", re.UNICODE)

In [45]:
def max_freq(tokens):
    w2cnt = Counter(tokens)
    max_value = max(w2cnt.items(), key=operator.itemgetter(1))[1]
    return max_value

In [23]:
doc_wordlist_pairs = doc_text_pairs.map(lambda x: (x[2],len(tokenize(x[0]))+len(tokenize(x[1]))))

DL = doc_wordlist_pairs.collectAsMap()



In [26]:
docid_title_pairs = doc_text_pairs.map(lambda x: (x[2],x[1]))

DT = docid_title_pairs.collectAsMap()

In [49]:
pv_clean = 'dl.pkl'
with open(pv_clean, 'wb') as f:
    pickle.dump(DL, f)

bucket = client.bucket(bucket_name)
blob = bucket.blob(f"dl/{pv_clean}")

blob.upload_from_filename(pv_clean)


In [50]:
pv_clean2 = 'dt.pkl'
with open(pv_clean2, 'wb') as f:
    pickle.dump(DT, f)

bucket = client.bucket(bucket_name)
blob = bucket.blob(f"dt/{pv_clean2}")

blob.upload_from_filename(pv_clean2)

In [51]:
# # print(doc_text_pairs.take(10))
# docid_maxfreq_pairs = doc_text_pairs.map(lambda x: (x[2],max_freq(tokenize(x[0]))))
# max_frequency=docid_maxfreq_pairs.collectAsMap()